# Try CGF! Test run is on a point in time where there's an eddy parked in the beam

In [ ]:
cd tasman-tides/

In [ ]:
import argparse
import ttidelib as tt
import os
import numpy as np
%matplotlib inline 
# os.chdir("tasman-tides")
import subprocess
import time
from dask.distributed import Client,default_client
from matplotlib import pyplot as plt
from pathlib import Path
home = Path("/home/149/ab8992/tasman-tides")
gdata = Path("/g/data/nm03/ab8992")
%matplotlib inline
import matplotlib
import xarray as xr
import filtering
def startdask():
    try:
    # Try to get the existing Dask client
        client = default_client()
        print(client)
    except ValueError:
        # If there's no existing client, create a new one
        client = Client()
        print(client)

startdask()

In [ ]:
import scipy
from ipywidgets import interact
import cmocean
cmap = cmocean.cm.dense_r
earth_cmap = matplotlib.cm.get_cmap("gist_earth")


def cross_scale_transfer(data,cutoff):
    sos = scipy.signal.butter(6,[cutoff],output = "sos")

    if "zl" in data.dims:
        dims = ["time","zl","yb","xb"]
    else:
        dims = ["time","yb","xb"]
    uu = scipy.signal.sosfilt(sos, data.u**2,axis = 0)
    vv = scipy.signal.sosfilt(sos, data.v**2,axis = 0)
    u = scipy.signal.sosfilt(sos, data.u,axis = 0)
    v = scipy.signal.sosfilt(sos, data.v,axis = 0)
    uv = scipy.signal.sosfilt(sos, data.u*data.v,axis = 0)

    out = xr.Dataset(
        {
            "tau_uu":(dims,uu - u**2),
            "tau_uv":(dims,uv - u*v),
            "tau_vv":(dims,vv - v**2),
            "u":(dims,u),
            "v":(dims,v),
            "uu":(dims,uu),
            "vv":(dims,vv)
        },
        coords = data.coords
    )


    transfer = (
        out.tau_uu * out.u.differentiate("xb") +
        out.tau_uv * out.u.differentiate("yb") +
        out.tau_uv * out.v.differentiate("xb") +
        out.tau_vv * out.v.differentiate("yb")
    ).rename("transfer")



    return xr.merge([out,transfer])




# tau = cross_scale_transfer(data)

## First read in a single that overlaps the eddy

In [ ]:
full = tt.collect_data(
    "full-20",
    rawdata = ["u","v"],
    timerange=(44850,45050)).isel(zl = 5)
full.u.attrs = {}
full.v.attrs = {}
full.zl.attrs = {}
full.time.attrs = {}
full.yb.attrs = {}
full.xb.attrs = {}



full = full.assign_coords({
    "time":full.time * 3600,
    "xb":full.xb * 1000,
    "yb":full.yb * 1000})
full = full.drop(["bathy","lat","lon"])

full.u.to_netcdf("/g/data/nm03/ab8992/postprocessed/full-20/usample.nc")
full.v.to_netcdf("/g/data/nm03/ab8992/postprocessed/full-20/vsample.nc")

# notide = xr.merge([
#     xr.open_mfdataset("/g/data/nm03/ab8992/outputs/notide-20/output1*/u/u*.nc",parallel = True,decode_times = False, decode_cf = False).u.sel(time = slice(37900,38100)).isel(zl = 5),
#     xr.open_mfdataset("/g/data/nm03/ab8992/outputs/notide-20/output1*/v/v*.nc",parallel = True,decode_times = False, decode_cf = False).v.sel(time = slice(37900,38100)).isel(zl = 5),
# ]).assign_coords({"time":full.time * 3600}).persist()
# wind_stress = xr.open_mfdataset("/g/data/nm03/ab8992/outputs/full-20/output*/surface_transect.nc",parallel = True,decode_times = False, decode_cf = False)[["taux","tauy"]].sel(time = slice(44900,45000))

## Test simple 12hr lagrange filter on this snapshot - same as test in the eulerian case

In [ ]:
os.getenv('PBS_JOBFS')

In [ ]:
m2freq = 2*np.pi/(12.42*3600)
m2freq

In [ ]:
from datetime import timedelta
import filtering
import numpy as np
import os 
if os.path.exists("/g/data/nm03/ab8992/postprocessed/full-20/waves.nc"):
    os.remove("/g/data/nm03/ab8992/postprocessed/full-20/waves.nc")
m2freq = 2*np.pi/(12.42*3600)
f = filtering.LagrangeFilter(
	"/g/data/nm03/ab8992/postprocessed/full-20/waves",
      {"U":"/g/data/nm03/ab8992/postprocessed/full-20/usample.nc","V":"/g/data/nm03/ab8992/postprocessed/full-20/vsample.nc"}, {"U":"u","V":"v"}, {"lon":"xb","lat":"yb","time":"time"},
	sample_variables=["U","V"], mesh="flat",highpass_frequency = m2freq,
    advection_dt =timedelta(hours=24).total_seconds(),
    window_size = timedelta(hours=24).total_seconds(),
)
f(times = range(3600 * 50,3600 * 150,3600))

In [ ]:
import netCDF4

netCDF4.Dataset("waves" + ".nc", "w", clobber=False)

In [ ]:
full_12f = cross_scale_transfer(full,1/12).persist()
notide_12f  = cross_scale_transfer(notide,1/12).persist()
full_6f = cross_scale_transfer(full,1/6).persist()
notide_6f  = cross_scale_transfer(notide,1/6).persist()
full_vorticity = tt.calculate_vorticity(full_12f.load()).persist()
notide_vorticity = tt.calculate_vorticity(notide_12f.load()).persist()
bathy = xr.open_mfdataset("/g/data/nm03/ab8992/outputs/full-20/bathy_transect.nc").elevation
bathy = bathy.where(bathy > 0).compute().rename("bathy")

data_full = xr.merge([full_12f.transfer.rename("energy_transfer"),full_vorticity.rename("vorticity"),bathy]).persist()
data_notide = xr.merge([notide_12f.transfer.rename("energy_transfer"),notide_vorticity.rename("vorticity"),bathy]).persist()

In [ ]:
def plot_transfer(data,vmax = 0.1):
    """
    data needs to contain bathy, energy transfer, b"""
    cmap = matplotlib.cm.get_cmap("RdBu")
    fig,ax = plt.subplots(2,figsize = (15,12))
    # Set the background colour to the plot to the lowest value in the cmap
    ax[0].set_facecolor("white")
    ax[1].set_facecolor("white")
    
    data.energy_transfer.plot(ax = ax[0],cmap = cmap,add_colorbar = True,vmax = vmax,vmin = -vmax,cbar_kwargs={'label': "Energy transfer (m^2/s^3)"})
    data.vorticity.plot(ax = ax[1],cmap = cmap,add_colorbar = True,cbar_kwargs={'label': "Vorticity"})

    data.bathy.plot(cmap = earth_cmap,vmin = -1000,vmax = 1500,ax = ax[0],add_colorbar = False)
    data.bathy.plot(cmap = earth_cmap,vmin = -1000,vmax = 1500,ax = ax[1],add_colorbar = False)

    ax[0].set_xlabel("")
    ax[0].set_title(f"Energy transfer from small to larger scales")
    ax[1].set_title(f"Vorticity")
    plt.show()
    return fig

# tt.make_movie(data_full,plot_transfer,"full-20","12hr_filtered_energy_transfer")
# plot_transfer(data_full,1/12,100)


In [ ]:
fn = "/g/data/v45/ab8992/processed_data/singleridge_10xwind/yextent_100"

variables = {"U": "u", "V": "v","phi":"phi"}
filenames = {v: fn for v in variables.keys()}
dimensions = {"lon": "xh", "lat": "yh", "time": "time", "depth": "zl"}

i = 1

path = "/g/data/v45/ab8992/processed_data/lagrangefilter"
name = "lagrange_filter_largedomain"+str(i)

if name + ".nc" in os.listdir(path):
    os.remove(path + "/" +  name + ".nc")
print(i)
f = filtering.LagrangeFilter(path + "/" +  name, filenames, variables, dimensions,
    sample_variables=["U","phi"], 
    mesh="flat",
    highpass_frequency = 1e-4,
    advection_dt =timedelta(minutes=5),
    window_size=timedelta(days=2).total_seconds(),indices={"depth": [i]}
)
f.seed_subdomain(min_lat=-8000, max_lat=8000)

f.make_zonally_periodic()
f()